In [ ]:
import importlib
itertools = importlib.import_module('ee_helpers')

import ee 
from datetime import datetime
from datetime import timedelta
import os
import geemap
import geopandas as gpd
from ee_helpers import EePull
import pandas as pd
import numpy as np
import rasterio

In [ ]:
# Trigger the authentication flow
ee.Authenticate()

# Initialize the library
ee.Initialize()

In [ ]:
data_label = "localized_time_series_for_training_c02_t1_l2"
batch_num = 1
glacier_view_dir = os.path.join(os.path.expanduser('~'),"Desktop","projects","GlacierView")
log_dir = os.path.join(glacier_view_dir,"src","earth_engine","data","ee_landing_zone",data_label, "logs")
image_dir = os.path.join(glacier_view_dir,"src","earth_engine","data","ee_landing_zone",data_label, "landsat")

In [ ]:
ee_pull = EePull(log_dir, "full_training_data")

In [ ]:
glims_bb_path = os.path.join(glacier_view_dir,"src","glims","data","training_sample","glims_18k_bb.shp")
df = gpd.read_file(glims_bb_path)

In [ ]:
df.bboxes[0]

In [ ]:
#only use if you want to resume training

done_df = pd.read_csv(os.path.join(log_dir,"training_log_1.log"), header = None, delimiter = ';')
done_glims_ids = list(np.unique([item.split(":")[-1] for item in done_df[0]]))

df = df[~df.glac_id.isin(done_glims_ids)]

In [ ]:
df

In [ ]:
df.shape

In [ ]:
len(done_glims_ids)

In [ ]:
df.shape

In [ ]:
#shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)
n = df.shape[0]

In [ ]:
for idx,row in enumerate(df.iterrows()):
    glims_id = row[1].glac_id
    bounding_box = eval(row[1].bboxes)
    date = row[1].src_date.split("T")[0]
    date = datetime.strptime(date, '%Y-%m-%d')
    out_dir = os.path.join(image_dir,glims_id)
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    
    start_date = date - timedelta(days = 50)
    end_date = date + timedelta(days = 50)
    
    try:
        ee_pull.export_landsat_five_images(glims_id,
                                            bounding_box,
                                            start_date,
                                            end_date,
                                            out_dir)
    except:
        pass
    
    try:
        ee_pull.export_landsat_seven_images(glims_id,
                                            bounding_box,
                                            start_date,
                                            end_date,
                                            out_dir)
    except:
        pass
    try:
        ee_pull.export_landsat_eight_images(glims_id,
                                            bounding_box,
                                            start_date,
                                            end_date,
                                            out_dir)
    except:
        pass

In [ ]:
os.mkdir("data/training/dems")
for row in df.iterrows():
    glims_id = row[1].glac_id
    bounding_box = eval(row[1].bboxes)
    out_dir = os.path.join("data","training","dems")
    try:
        ee_helpers.export_nasa_dems(glims_id, bounding_box, out_dir)
    except:
        pass

In [ ]:
pd.Series(completed).to_csv("completed_batch_1.csv")